In [4]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [14]:
#loading images in
cwd = os.getcwd()#return current working directory
patient_id = os.listdir('ADNI') #create a list that encompasses all the images we downloaded
filename = glob.glob('data/' + '*/data_batch*')

#import csv files
datacsv = np.genfromtxt('ADNI1_Complete_1Yr_3T_11_04_2018.csv', delimiter = ',', encoding = None, dtype = None)

#modify csv files
datacsv = np.delete(datacsv, 0, 1)
datacsv = np.delete(datacsv, np.s_[4:12], 1)
datacsv = np.delete(datacsv, 0, 0)
    #rename csv group into numbers
for i in range(datacsv.shape[0]):
    if datacsv[i][1] == '"MCI"':
        datacsv[i][1] = 0
    elif datacsv[i][1] == '"AD"':
        datacsv[i][1] = 1
    elif datacsv[i][1] == '"CN"':
        datacsv[i][1] = 2
#filter out repeated patient in csv by saving their first sMRI and its corresponding infos
pat_id = datacsv[0][0]
flag_0 = 0
datacsv_new = datacsv.copy()#[patient_id, group, sex, age]
deleted_line = 0
for i in range(datacsv.shape[0]):
    if (pat_id == datacsv[i][0]) & (flag_0 >= 1):
        datacsv_new = np.delete(datacsv_new, i - deleted_line, 0)
        deleted_line = deleted_line + 1
    elif (pat_id == datacsv[i][0]) & (flag_0 == 0):
        flag_0 = flag_0 + 1
    elif (pat_id != datacsv[i][0]):
        flag_0 = 1
        pat_id = datacsv[i][0]
        
        
#Gaining sMRI        
    #datapath: obtain the path to reach each patient's sMRI 
datapath = [] #[patient_id, path]
for patient in patient_id:
    flag = 0
    for root, dirs, files in os.walk(cwd + "/ADNI/" + patient):         
        for file in files:
            if file.endswith(".nii"):
                if flag < 1:
                    datapath.append([patient, os.path.join(root, file)])
                    flag = flag + 1
                    
    #sMRIs were loaded and reshaped into the targeted dim[256, 256, 160] 
print(datapath)

[['136_S_1227', '/home/ubuntu/working/ADNI/136_S_1227/MPR____N3__Scaled/2007-02-21_13_12_42.0/S26837/ADNI_136_S_1227_MR_MPR____N3__Scaled_Br_20070810000731580_S26837_I66824.nii'], ['002_S_1018', '/home/ubuntu/working/ADNI/002_S_1018/MPR____N3__Scaled/2007-12-07_18_28_23.0/S43496/ADNI_002_S_1018_MR_MPR____N3__Scaled_Br_20080313095739932_S43496_I97022.nii'], ['127_S_0260', '/home/ubuntu/working/ADNI/127_S_0260/MPR__GradWarp__B1_Correction__N3__Scaled/2007-05-08_13_08_10.0/S31931/ADNI_127_S_0260_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070731195531372_S31931_I63565.nii'], ['023_S_0061', '/home/ubuntu/working/ADNI/023_S_0061/MPR__GradWarp__B1_Correction__N3__Scaled/2006-06-15_13_13_25.0/S15579/ADNI_023_S_0061_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070804141852672_S15579_I64681.nii'], ['031_S_1209', '/home/ubuntu/working/ADNI/031_S_1209/MPR__GradWarp__B1_Correction__N3__Scaled/2008-02-08_13_06_55.0/S45594/ADNI_031_S_1209_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20080

In [11]:
dataimage = []#[patient_id, image_matrix_normalized]
tar_dim = [256, 256, 160]
for path in datapath:
    mri = nib.load(path[1]).get_data()
    mri = (mri - mri.min())/(mri.max() - mri.min())
    mri = mri - mri.mean()
    tar_mri = ndimage.zoom(mri,[tar_dim[0]/mri.shape[0],tar_dim[1]/mri.shape[1], tar_dim[2]/mri.shape[2]], order = 1)
    dataimage.append([path[0],tar_mri])

dataimage = sorted(dataimage, key=itemgetter(0), reverse=True)


<class 'numpy.ndarray'>


In [ ]:
#save the sMRIs and their corresponding csv file (both in txt format) to the GPU
with open("dataimage_arr.txt", "wb") as fp:
    pickle.dump(dataimage, fp)
with open("datacsv_new_arr.txt", "wb") as fp:
    pickle.dump(datacsv_new, fp)